In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#import basic essential libraries 
import matplotlib.pyplot as plt
import path
import os
%matplotlib inline

In [ ]:
# import  keras libraries to build model and conv net
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from pathlib import Path

# load dataset and image agumentaion 

In [ ]:
# current dataset only has 4k images
# so image agumentation to virtually increase size of dataset 
# help (ImageDataGenerator?)

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

data_agumentation=ImageDataGenerator(rescale=0./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True

)

# load training data
train_data=data_agumentation.flow_from_directory(directory='../input/indian-currency-note-images-dataset-2020/Indian currency dataset v1/training',
                                                 target_size=(256,256),
                                                 class_mode='categorical',
                                                batch_size=32 
                                               )

In [ ]:
#load validation data
val_data=ImageDataGenerator().flow_from_directory(directory='../input/indian-currency-note-images-dataset-2020/Indian currency dataset v1/validation',
                                                      target_size=(256,256),
                                                       class_mode='categorical'
                                                      )

# Model 

### Early stopping and callback

In [ ]:
# import modelcheckpoint and earlystopping  

from tensorflow.python.keras.callbacks import ModelCheckpoint,EarlyStopping

# checkpoint monitors  the given parameter and save the model automatically
# here given parameter to monitor is val_loss
# it moniters the  val loss of each epoch and val_loss is lower than previous one it save the current model ad wieght

checkpoint=ModelCheckpoint("currency_detector_smal_model.h5", monitor='val_loss',verbose=1, save_best_only=True,save_weights_only=False,mode='auto', period=1)

# early stopping .. it stops the trainng phase if there is no improvement in the model
# patience defines how many epoch can b ignored before forcefully stoping the model

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

### define model add layers and compile

In [ ]:
# import ResNet50

from tensorflow.keras.applications import ResNet50

# total number of classes (7 diffferent currency and 1 background class)
num_classes = 8

# load the weight
resnet_weights_path = '../input/resnet-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

#define the mdel
my_new_model = Sequential()

# add the resnet to new defined model
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))

#last fully connected layer with softmax as a activation function
my_new_model.add(Dense(num_classes, activation='softmax'))

my_new_model.layers[0].trainable = False

In [ ]:
# compile the model with adam optimizer, categorical_croosentropy loss function
my_new_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Fit the model with train and val dataset

In [ ]:
# fit the model with train data and validation data 
# epoch 50
my_new_model.fit_generator(
        train_data,
        epochs = 50,
        validation_data=val_data,
        callbacks=[checkpoint,early])

It almost takes 4 - 4.5 minutes for each epoch.....

i already trained the model ... 

it stops after 40 epochs due to early stopping ...

it will takes almost 2. 5 hrs to train so i will cancel this training phase...

No file is in output dorectory... after one epoch the weight file will generate and save in output directory..
for each imporvement the same weight file will get updated with the new weights...
wait for the first epoch to finish.
..
 the wights file after  1st epoch  is saved...
 
 i will cancel this training now 
 




### save model into JSON

In [ ]:
# save the json model  

model_json = my_new_model.to_json()
with open("resnet_50_model.json", "w") as json_file:
    json_file.write(model_json)

# predication

### class labels 

In [ ]:
# These are the  class labels from the training data (Each number stands for the currency denomination)
class_labels = [
    '10','100','20','200','2000','50','500','Background'
]

### Show images 

In [ ]:
# Load an image file to test, resizing it to 256x256 pixels
# to save time in training I resize images to 256x256 

from tensorflow.python.keras.preprocessing import image

img = image.load_img("../input/indian-currency-notes/indian_currency_new/validation/100/38.jpg", target_size=(256,256))
plt.imshow(img)

 ### code for prediction

In [ ]:
# Convert the image to a numpy array
from tensorflow.python.keras.preprocessing import image

def prediction(file_name):
    img = image.load_img(file_name, target_size=(256,256))

    image_to_test = image.img_to_array(img)

    # Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
    list_of_images = np.expand_dims(image_to_test, axis=0)

    # Make a prediction using the model
    results = my_new_model.predict(list_of_images)

    # Since we are only testing one image, we only need to check the first result
    single_result = results[0]

    # We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
    most_likely_class_index = int(np.argmax(single_result))
    class_likelihood = single_result[most_likely_class_index]

    # Get the name of the most likely class
    class_label = class_labels[most_likely_class_index]

    # Print the result
    print(file_name)
    print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

### predicting the test images 

In [ ]:


from tensorflow.python.keras.preprocessing import image
img = image.load_img("../input/indian-currency-notes/indian_currency_new/validation/100/38.jpg", target_size=(256,256))
plt.imshow(img)

In [ ]:
import glob
# Find all *.jpg files in the directory
file_name_list = glob.glob('../input/indian-currency-notes/indian_currency_new/validation/100/38.jpg')
print(len(file_name_list))
for file_name in file_name_list:
    prediction(file_name)